In [2]:
import pandas as pd
import numpy as np

# loading in the training and test datasets
train_data = pd.read_csv('dmt-2025-2nd-assignment/training_set_VU_DM.csv')
test_data = pd.read_csv('dmt-2025-2nd-assignment/test_set_VU_DM.csv')

In [3]:
# The assignment said there are nearly 5 million observations, verifying
print(train_data.shape) # (4958347, 54)
print(test_data.shape)  # (4959183, 50)
  
# The training data contains four columns that are not in the testing data. I will look at observations only where a booking was made
print(train_data[train_data['gross_bookings_usd'].notna()][['position', 'click_bool', 'booking_bool', 'gross_bookings_usd']].head(10))

# What about observations where no booking was made?
print(train_data[train_data['gross_bookings_usd'].isna()][['position', 'click_bool', 'booking_bool', 'gross_bookings_usd']].head(10))

(4958347, 54)
(4959183, 50)
     position  click_bool  booking_bool  gross_bookings_usd
12         13           1             1              114.29
63          1           1             1              162.38
68         16           1             1               96.41
194         4           1             1              222.58
211        22           1             1               47.10
243        26           1             1              286.32
274         1           1             1             1927.64
312        13           1             1             1831.30
346         4           1             1              226.76
387         1           1             1               60.77
   position  click_bool  booking_bool  gross_bookings_usd
0        27           0             0                 NaN
1        26           0             0                 NaN
2        21           0             0                 NaN
3        34           0             0                 NaN
4         4           